In [1]:
from datetime import datetime
import os
import json
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.utils import to_categorical, Sequence
from keras.models import Model
from keras.layers import (
    Input,
    Conv2D,
    MaxPooling2D,
    Flatten,
    Dense,
    Dropout,
    concatenate,
    BatchNormalization,
)
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import classification_report, confusion_matrix
from modules.PostgresDBHandler import PostgresDBHandler


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\Shengan\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\Shengan\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\Shengan\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\Shengan\AppData\Roaming\Python\Python310\site-packages\traitlets\config\ap

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\Shengan\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\Shengan\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\Shengan\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\Shengan\AppData\Roaming\Python\Python310\site-packages\traitlets\config\ap

AttributeError: _ARRAY_API not found

ImportError: numpy.core._multiarray_umath failed to import

ImportError: numpy.core.umath failed to import

TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle

In [2]:
gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            print("Number of avaliable GPUs: ", len(gpus))
    except RuntimeError as e:
        print(e)

Number of avaliable GPUs:  1


In [3]:
DB_PARAMS = {
    "dbname": "mydatabase",
    "user": "postgres",
    "password": "79eRReroWY",
    "host": "localhost",
    "port": "5432",
}
EPOCHS = 100
BATCH_SIZE = 128
KFOLD_SPLITS = 5

In [4]:
dbConnect = PostgresDBHandler(**DB_PARAMS)
dbConnect.connect()

In [5]:
instruments_mappings = dbConnect.get_mappings_instruments()

In [6]:
processedIDs = dbConnect.get_all_processed_ids()
# processedIDs = processedIDs[50:120]

processed_data = dbConnect.get_processed_fit_data(processedIDs)

In [7]:
mfcc_paths = [item["mfccPath"] for item in processed_data]
spectrogram_paths = [item["spectrogramPath"] for item in processed_data]
instrument_ids = [item["instrumentID"] for item in processed_data]
mappings = dbConnect.get_mappings_instruments().to_dict()['name']

In [8]:
processed_df = pd.DataFrame(
    {
        "mfccPath": mfcc_paths,
        "spectrogramPath": spectrogram_paths,
        "instrumentID": instrument_ids,
    }
)

In [9]:
dbConnect.close()

In [10]:
processed_df

,mfccPath,spectrogramPath,instrumentID
0,intermediate_results\mfcc\60e9784a-ac54-4bff-9...,intermediate_results\melSpectrogram\60e9784a-a...,8
1,intermediate_results\mfcc\306c2c6b-ffd3-4c2b-a...,intermediate_results\melSpectrogram\306c2c6b-f...,8
2,intermediate_results\mfcc\c21ef91a-6021-46b4-9...,intermediate_results\melSpectrogram\c21ef91a-6...,8
3,intermediate_results\mfcc\d85f01a8-9a6a-4c38-9...,intermediate_results\melSpectrogram\d85f01a8-9...,8
4,intermediate_results\mfcc\9104c033-ebcf-434d-8...,intermediate_results\melSpectrogram\9104c033-e...,8
...,...,...,...
65235,intermediate_results\mfcc\d5ba8934-0b92-4f88-8...,intermediate_results\melSpectrogram\d5ba8934-0...,7
65236,intermediate_results\mfcc\e6e708bb-a0bb-4af6-b...,intermediate_results\melSpectrogram\e6e708bb-a...,7
65237,intermediate_results\mfcc\91dd5af5-894d-4e50-8...,intermediate_results\melSpectrogram\91dd5af5-8...,7
65238,intermediate_results\mfcc\ac6691a9-4d91-45a4-b...,intermediate_results\melSpectrogram\ac6691a9-4...,7


In [11]:
class DataGenerator(Sequence):
    def __init__(
        self,
        df,
        fixed_length=128,
        batch_size=32,
        shuffle=True,
    ):
        self.df = df
        self.fixed_length = fixed_length
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.label_encoder = LabelEncoder()
        self.df.loc[:, "instrumentID"] = self.label_encoder.fit_transform(
            self.df["instrumentID"]
        )
        self.num_classes = len(self.label_encoder.classes_)
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.df) / self.batch_size))

    def __getitem__(self, index):
        indices = self.indices[index * self.batch_size : (index + 1) * self.batch_size]
        batch_df = self.df.iloc[indices]
        X, y = self.__data_generation(batch_df)
        return X, y

    def on_epoch_end(self):
        self.indices = np.arange(len(self.df))
        if self.shuffle:
            np.random.shuffle(self.indices)

    def __data_generation(self, batch_df):
        X_mels = []
        X_mfccs = []
        y = []

        for idx, row in batch_df.iterrows():
            mel_spectrogram = np.load(row["spectrogramPath"])
            mfccs = np.load(row["mfccPath"])

            X_mels.append(mel_spectrogram)
            X_mfccs.append(mfccs)
            y.append(row["instrumentID"])

        X_mels = np.expand_dims(np.array(X_mels), -1)
        X_mfccs = np.expand_dims(np.array(X_mfccs), -1)
        y = np.array(y)

        return (
            {"mel_input": X_mels, "mfccs_input": X_mfccs},
            to_categorical(y, num_classes=self.num_classes),
        )

In [12]:
def create_model(input_shape_mel, input_shape_mfccs, num_classes):    
    # Mel spectrogram branch
    input_mel = Input(shape=input_shape_mel, name="mel_input")
    x_mel = Conv2D(64, (3, 3), activation="relu")(input_mel)
    x_mel = MaxPooling2D((2, 2))(x_mel)
    x_mel = Conv2D(128, (3, 3), activation="relu")(x_mel)
    x_mel = MaxPooling2D((2, 2))(x_mel)
    x_mel = Conv2D(256, (3, 3), activation="relu")(x_mel)
    x_mel = MaxPooling2D((2, 2))(x_mel)
    x_mel = Flatten()(x_mel)

    # MFCC branch
    input_mfccs = Input(shape=input_shape_mfccs, name="mfccs_input")
    x_mfccs = Conv2D(64, (3, 3), activation="relu")(input_mfccs)
    x_mfccs = MaxPooling2D((2, 2))(x_mfccs)
    x_mfccs = Conv2D(128, (3, 3), activation="relu")(x_mfccs)
    x_mfccs = MaxPooling2D((2, 2))(x_mfccs)
    x_mfccs = Flatten()(x_mfccs)

    # Concatenate the outputs of both branches
    concatenated = concatenate([x_mel, x_mfccs])
    # Fully connected layers
    x = Dense(64, activation="relu")(concatenated)
    x = Dropout(0.25)(x)
    x = BatchNormalization()(x)
    output = Dense(num_classes, activation="softmax")(x)

    model = Model(inputs=[input_mel, input_mfccs], outputs=output)
    return model

In [13]:
input_shape_mel = (128, 128, 1)  
input_shape_mfccs = (13, 128, 1)  
num_classes = len(np.unique(processed_df["instrumentID"]))

In [14]:
early_stopping = EarlyStopping(
    monitor="val_loss", patience=10, restore_best_weights=True
)

# Cross validation

In [15]:
accuracy_list = []
loss_list = []
classification_reports = []
confusion_matrices = []
history_list = []

In [16]:
kf = KFold(n_splits=KFOLD_SPLITS, shuffle=True)

In [17]:
for train_index, cross_index in kf.split(processed_df):

    # Split the data
    X_train, X_val = train_test_split(
        processed_df.iloc[train_index], test_size=0.2, shuffle=True
    )

    # Create data generators
    train_generator = DataGenerator(X_train, batch_size=350)
    val_generator = DataGenerator(X_val, batch_size=350)
    cross_generator = DataGenerator(processed_df.iloc[cross_index], batch_size=350, shuffle=False)

    # Create and compile the model
    model = create_model(input_shape_mel, input_shape_mfccs, num_classes)
    
    optimizer = tf.keras.optimizers.Adam(lr = 0.0005)
    
    model.compile(
        optimizer= optimizer,
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )

    # Train the model
    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=EPOCHS,
        callbacks=[early_stopping],
    )
    
    history_list.append(history.history)

    # Evaluate the model
    loss, accuracy = model.evaluate(cross_generator)
    accuracy_list.append(accuracy)
    loss_list.append(loss)

    # Predict using the model
    y_pred = model.predict(cross_generator)
    y_pred_classes = np.argmax(y_pred, axis=1)

    # Extract true labels from the generator
    y_true = []
    for _, labels in cross_generator:
        y_true.extend(np.argmax(labels, axis=1))
    y_true = np.array(y_true)

    # Generate classification report and confusion matrix
    report = classification_report(y_true, y_pred_classes, output_dict=True)
    classification_reports.append(report)

    conf_matrix = confusion_matrix(y_true, y_pred_classes).tolist()
    confusion_matrices.append(conf_matrix)

c:\Users\Shengan\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/100
119/119 [==============================] - 38s 256ms/step - loss: 1.6814 - accuracy: 0.4524 - val_loss: 1.5050 - val_accuracy: 0.6826
Epoch 2/100
119/119 [==============================] - 19s 158ms/step - loss: 0.8575 - accuracy: 0.8152 - val_loss: 1.0468 - val_accuracy: 0.8670
Epoch 3/100
119/119 [==============================] - 19s 155ms/step - loss: 0.5526 - accuracy: 0.8831 - val_loss: 0.6470 - val_accuracy: 0.9282
Epoch 4/100
119/119 [==============================] - 18s 154ms/step - loss: 0.4028 - accuracy: 0.9172 - val_loss: 0.3848 - val_accuracy: 0.9309
Epoch 5/100
119/119 [==============================] - 19s 155ms/step - loss: 0.3108 - accuracy: 0.9342 - val_loss: 0.2609 - val_accuracy: 0.9555
Epoch 6/100
119/119 [==============================] - 19s 155ms/step - loss: 0.2581 - accuracy: 0.9424 - val_loss: 0.2239 - val_accuracy: 0.9560
Epoch 7/100
119/119 [==============================] - 18s 151ms/step - loss: 0.2055 - accuracy: 0.9562 - val_loss: 0.1433 -

c:\Users\Shengan\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


119/119 [==============================] - 19s 156ms/step - loss: 1.1999 - accuracy: 0.6236 - val_loss: 0.8991 - val_accuracy: 0.8057
Epoch 2/100
119/119 [==============================] - 18s 154ms/step - loss: 0.4158 - accuracy: 0.9002 - val_loss: 0.4264 - val_accuracy: 0.9323
Epoch 3/100
119/119 [==============================] - 19s 155ms/step - loss: 0.2535 - accuracy: 0.9398 - val_loss: 0.2256 - val_accuracy: 0.9527
Epoch 4/100
119/119 [==============================] - 19s 155ms/step - loss: 0.1841 - accuracy: 0.9556 - val_loss: 0.1708 - val_accuracy: 0.9639
Epoch 5/100
119/119 [==============================] - 19s 155ms/step - loss: 0.1496 - accuracy: 0.9635 - val_loss: 0.1280 - val_accuracy: 0.9644
Epoch 6/100
119/119 [==============================] - 19s 155ms/step - loss: 0.1140 - accuracy: 0.9717 - val_loss: 0.0973 - val_accuracy: 0.9770
Epoch 7/100
119/119 [==============================] - 19s 155ms/step - loss: 0.0930 - accuracy: 0.9774 - val_loss: 0.0762 - val_accurac

c:\Users\Shengan\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


119/119 [==============================] - 19s 154ms/step - loss: 1.2277 - accuracy: 0.6273 - val_loss: 0.9523 - val_accuracy: 0.7925
Epoch 2/100
119/119 [==============================] - 18s 154ms/step - loss: 0.5107 - accuracy: 0.8828 - val_loss: 0.5296 - val_accuracy: 0.9171
Epoch 3/100
119/119 [==============================] - 18s 154ms/step - loss: 0.3151 - accuracy: 0.9286 - val_loss: 0.2860 - val_accuracy: 0.9541
Epoch 4/100
119/119 [==============================] - 18s 154ms/step - loss: 0.2310 - accuracy: 0.9463 - val_loss: 0.2165 - val_accuracy: 0.9557
Epoch 5/100
119/119 [==============================] - 18s 154ms/step - loss: 0.1702 - accuracy: 0.9602 - val_loss: 0.1559 - val_accuracy: 0.9613
Epoch 6/100
119/119 [==============================] - 18s 154ms/step - loss: 0.1356 - accuracy: 0.9688 - val_loss: 0.1283 - val_accuracy: 0.9645
Epoch 7/100
119/119 [==============================] - 18s 153ms/step - loss: 0.1142 - accuracy: 0.9733 - val_loss: 0.1241 - val_accurac

c:\Users\Shengan\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


119/119 [==============================] - 71s 596ms/step - loss: 1.0340 - accuracy: 0.6878 - val_loss: 0.6666 - val_accuracy: 0.8823
Epoch 2/100
119/119 [==============================] - 20s 169ms/step - loss: 0.3874 - accuracy: 0.9091 - val_loss: 0.4140 - val_accuracy: 0.9327
Epoch 3/100
119/119 [==============================] - 19s 163ms/step - loss: 0.2406 - accuracy: 0.9438 - val_loss: 0.2086 - val_accuracy: 0.9508
Epoch 4/100
119/119 [==============================] - 20s 163ms/step - loss: 0.1730 - accuracy: 0.9585 - val_loss: 0.1526 - val_accuracy: 0.9619
Epoch 5/100
119/119 [==============================] - 20s 163ms/step - loss: 0.1334 - accuracy: 0.9690 - val_loss: 0.1069 - val_accuracy: 0.9757
Epoch 6/100


KeyboardInterrupt: 

# Save raports and model

In [ ]:
try:
    os.mkdir("models")
except FileExistsError:
        print("Folder already exists")
except Exception:
    print("Unknown error")
    
model.save("models/instrument_classifier_model.h5")


## Create training version folder

In [ ]:
date_part = datetime.now().date().__str__().replace('-', '_')
last_version = os.listdir(path="models")
last_version = [name.rpartition("_v")[-1] for name in last_version if date_part in name]
if len(last_version):
    last_version = int(sorted(last_version)[-1])
else:
    last_version = 0
folder_name = f"{date_part}_v{last_version+1}"

In [44]:
os.mkdir(os.path.join("models", folder_name))

## Store data

In [45]:
raport = {
    "accuracy_list": accuracy_list,
    "loss_list": loss_list,
    "classification_reports": classification_reports,
    "confusion_matrices": confusion_matrices,
    "histories": history_list,
    "mappings": mappings
}

In [48]:
with open(os.path.join("models", folder_name, "raport.json"), "w")as raportFile:
    json.dump(raport, raportFile)